KÜTÜPHANE YÖNETİM SİSTEMİ (LMS) FİNAL PROJE RAPORU
Öğrenci Adı: [Adın Soyadın] Numara: [Numaran] Ders: Veritabanı Yönetim Sistemleri / Yazılım Mühendisliği Tarih: [Tarih]

1. GERÇEK DÜNYA PROBLEMİNİN TANIMI
Günümüzde bilgiye erişimin en temel noktalarından biri olan kütüphanelerde, binlerce kitabın ve yüzlerce kullanıcının hareketlerinin (ödünç alma, iade etme, stok takibi) manuel yöntemlerle veya eski teknolojilerle takip edilmesi büyük sorunlara yol açmaktadır.

Geleneksel yöntemlerde (defter tutma veya Excel tabloları):

Hangi kitabın kimde olduğunu bulmak zaman almaktadır.
İade tarihi geçen kitapların takibi zorlaşmaktadır.
Kitap stok durumlarında tutarsızlıklar yaşanmaktadır.
Veri kaybı riski yüksektir ve raporlama yapmak imkansıza yakındır.
Bu proje, bu problemleri çözmek amacıyla, verilerin merkezi bir veritabanında tutulduğu, tutarlılığın sağlandığı ve işlemlerin web tabanlı bir arayüz üzerinden hızlıca yapılabildiği bir Kütüphane Yönetim Sistemi geliştirmeyi hedefler.

2. GEREKSİNİM ANALİZİ
Sistemin başarılı olabilmesi için aşağıdaki gereksinimlerin karşılanması belirlenmiştir:

2.1. Fonksiyonel Gereksinimler
Kullanıcı Yönetimi: Sisteme "Öğrenci" ve "Personel" (Admin) olmak üzere iki tür kullanıcı giriş yapabilmelidir.
Kitap İşlemleri: Personel yeni kitap ekleyebilmeli, silebilir ve güncelleyebilmelidir.
Arama: Kullanıcılar kitap adına, yazarına veya yayıncısına göre arama yapabilmelidir.
Ödünç Alma/İade: Stokta olan kitaplar ödünç alınabilmeli, iade edildiğinde stok güncellenmelidir.
Stok Kontrolü: Stok sayısı 0 olan kitapların ödünç verilmesi sistem tarafından engellenmelidir.
2.2. Fonksiyonel Olmayan Gereksinimler
Veri Bütünlüğü: Bir kitap silindiğinde, geçmiş ödünç kayıtlarında tutarsızlık oluşmamalıdır.
Performans: Binlerce kitap arasından arama işlemi 1 saniyenin altında gerçekleşmelidir.
Kullanılabilirlik: Arayüz sade ve anlaşılır olmalıdır.
3. KAVRAMSAL TASARIM (EER DİYAGRAMLARI)
Veritabanı tasarımının ilk aşamasında varlıklar ve aralarındaki ilişkiler belirlenmiştir.

Temel Varlıklar:

User (Kullanıcı): ID, Kullanıcı Adı, Şifre, Rol (Öğrenci/Personel), Email.
Book (Kitap): ID, Kitap Adı, Yazar, Yayın Yılı, Yayıncı, Stok Sayısı, Resim URL.
BorrowTransaction (Ödünç İşlemi): ID, Alış Tarihi, İade Tarihi (Tahmini), İade Tarihi (Gerçekleşen).
İlişkiler:

Bir Kullanıcı birden fazla Kitap ödünç alabilir (1:N).
Bir Kitap birden fazla Kullanıcı tarafından (farklı zamanlarda) ödünç alınabilir (N:M).
Bu çoktan çoğa ilişki, BorrowTransaction adlı ilişki tablosu ile çözümlenmiştir.
[BURAYA EER DİYAGRAMI GÖRSELİNİ EKLEYİNİZ] (Not: Draw.io veya benzeri bir araçla çizdiğiniz; User, Book kutucukları ve aralarındaki baklava dilimi şeklindeki ilişkileri gösteren diyagram.)

4. MANTIKSAL TASARIM VE ŞEMA DİYAGRAMLARI
Kavramsal tasarım, 3. Normal Form (3NF) kurallarına uygun olarak ilişkisel şemaya dönüştürülmüştür.

Tablo Şemaları:

CustomUser Tablosu:

id (PK, Integer, Auto Increment)
username (Varchar, Unique)
email (Varchar)
is_staff (Boolean)
Book Tablosu:

id (PK, Integer, Auto Increment)
title (Varchar)
author (Varchar)
publisher (Varchar)
year (Integer)
stock (Integer)
Borrowing Tablosu:

id (PK, Integer)
user_id (FK -> CustomUser.id)
book_id (FK -> Book.id)
borrow_date (Date/Time)
return_date (Date/Time, Nullable)
[BURAYA TABLOLARI VE OKLARLA BAĞLANTILARINI GÖSTEREN ŞEMA GÖRSELİNİ EKLEYİNİZ]

5. TASARIMIN UYARLANMASI
5.1. Mimari ve Ortam Bilgileri
Uygulama 3 Katmanlı Mimari (3-Tier Architecture) prensibine göre geliştirilmiştir:

Sunum Katmanı (Presentation Layer): HTML5, CSS3 ve Django Template Engine.
Uygulama Katmanı (Application Layer): Python programlama dili ve Django Web Framework.
Veri Katmanı (Data Layer): SQLite Veritabanı.
Veritabanı Bağlantısı: Uygulama, veritabanına erişim için Python'ın DB-API standartlarını (JDBC/ODBC mantığına benzer şekilde) kullanan Django ORM (Object Relational Mapping) katmanını kullanır. Bu sayede SQL sorguları doğrudan yazılmak yerine Python nesneleri üzerinden yönetilir, bu da güvenlik (SQL Injection koruması) ve veritabanı bağımsızlığı sağlar.

5.2. Tabloların Oluşturulması (DDL)
Django ORM kullanılarak modeller oluşturulmuş ve migrate komutu ile veritabanında aşağıdaki SQL karşılıklarına benzer tablolar yaratılmıştır:

SQL
CREATE TABLE "library_book" (
    "id" integer NOT NULL PRIMARY KEY AUTOINCREMENT,
    "title" varchar(200) NOT NULL,
    "author" varchar(100) NOT NULL,
    "stock" integer NOT NULL
);
5.3. Veri Yükleme (Data Loading)
Projenin en kritik aşamalarından biri büyük veri setlerinin sisteme entegrasyonudur.

Kaynak: TurkishBookDataSet.csv dosyası.
Yöntem: Python'un Pandas kütüphanesi kullanılarak CSV dosyası okunmuş (book_load.py), veriler temizlenmiş (NaN kontrolü) ve döngüsel olarak veritabanına kaydedilmiştir.
Komut: python book_load.py
5.4. İndisler (Indexes)
Sorgu performansını artırmak için aşağıdaki alanlarda İndeks (Index) kullanılmıştır:

Primary Key alanlarında (id) otomatik indeksleme.
Sık arama yapılan Book.title ve Book.author alanlarında arama hızını artırmak için B-Tree yapısında indeks tanımlanmıştır.
5.5. Sorgu Tasarımları (Queries)
Sistem içinde kullanılan temel sorgu mantıkları şöyledir:

Stokta olan kitapları listeleme (SELECT): SELECT * FROM library_book WHERE stock > 0;
Kitap Ödünç Alma (INSERT & UPDATE):
INSERT INTO library_borrowing (user_id, book_id, date) VALUES (1, 5, NOW());
UPDATE library_book SET stock = stock - 1 WHERE id = 5;
6. UYGULAMA PROGRAM TANITIM VE ÖRNEK KULLANIM
Uygulama arayüzü kullanıcı dostu olarak tasarlanmıştır.

Senaryo 1: Kitap Arama ve Ödünç Alma Kullanıcı sisteme giriş yapar. Ana sayfadaki arama çubuğuna "Nutuk" yazar. Sonuçlarda kitabın stok durumu "Mevcut" olarak görünür. "Ödünç Al" butonuna tıklar. Sistem başarı mesajı döner ve stok 1 azalır.

Senaryo 2: Yönetici Paneli Admin kullanıcısı /admin adresine gider. "Add Book" butonuna tıklar. Yeni gelen kitapların bilgilerini girer ve kaydeder.

[BURAYA PROJENİN EKRAN GÖRÜNTÜLERİNİ EKLEYİNİZ] (Örn: Ana sayfa, Kitap Detay Sayfası, Admin Paneli görüntüsü)

7. SONUÇ
Bu proje kapsamında, gerçek bir kütüphane ortamının ihtiyaçlarını karşılayacak ölçeklenebilir bir Kütüphane Yönetim Sistemi başarıyla geliştirilmiştir. Pandas ile toplu veri yükleme yeteneği sayesinde sistem boş bir veritabanı ile değil, binlerce kitapla kullanıma hazır hale getirilmiştir. İlişkisel veritabanı tasarımı sayesinde veri tekrarları önlenmiş ve tutarlılık sağlanmıştır.

İleriki çalışmalarda sisteme; barkod okuyucu entegrasyonu ve yapay zeka tabanlı "Kullanıcıya Kitap Önerme" modülü eklenmesi planlanmaktadır.

8. REFERANSLAR
Django Software Foundation. "Django Documentation". https://docs.djangoproject.com/
Python Pandas Documentation. "User Guide". https://pandas.pydata.org/
Elmasri, R., & Navathe, S. (2016). Fundamentals of Database Systems. Pearson.
Proje Veri Seti: Turkish Book Dataset (Kaggle/GitHub).